In [1]:
import os
import sys
import rdkit
import svgutils.transform as sg

from rdkit.Chem.Draw import rdMolDraw2D
from jazzy import core, visualisation, helpers
# from cairosvg import svg2png

In [2]:
# Import config
opt_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'optimisation'))
sys.path.insert(0, opt_path)
import config

# Open the compressed data
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', config.DATA_PATH))

In [3]:
# Get all molecules in folder
cdk2_path = os.path.join(data_path, "cdk2_analysis")
files = os.listdir(cdk2_path)
files = [f for f in files if f.endswith("sdf") or f.endswith("mol")]
files

['20Z.sdf', '26Z.sdf', 'X02.sdf', 'X35.sdf', 'X36.sdf', 'X44.sdf']

In [4]:
def atomistic_strength_from_file(file_path):
    """Accepts a file path to an SDF/MOL file and generates an HB strength visualisation.
    """
    rdkit_mol = rdkit.Chem.MolFromMolFile(file_path)
    rdkit_mol = rdkit.Chem.AddHs(rdkit_mol, addCoords=True)
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    img_text = visualisation.depict_strengths(rdkit_mol, 
                                            atomic_map, 
                                            fig_size=(500, 500),
                                            flatten_molecule=True, 
                                            highlight_atoms=True, 
                                            ignore_sdc=True, 
                                            ignore_sdx=False,
                                            ignore_sa=True,
                                            sdc_threshold=0.0, 
                                            sdx_threshold=0.0,
                                            sa_threshold=0.0,
                                            rounding_digits=2)

    # Add ligand name as a title
    img_text = img_text.replace('svg:','')
    fig = sg.fromstring(img_text)
    label = sg.TextElement(250, 15, file_path.split("/")[-1], size=14, 
                        font='sans-serif', anchor='middle', color='#000000')
    fig.append(label)
    img_text = fig.to_str().decode("utf-8")
    return img_text

In [5]:
# Build strength visualisations
strengths = list()
for f in files:
    file_path = os.path.join(cdk2_path, f)
    strengths.append(atomistic_strength_from_file(file_path))

In [6]:
class HorizontalDisplay:
    """
    Accepts a list of SVGs, concatenates them, and returns a horizonal rendering.
    """
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        concat_svgs = ''.join(self.args[0])
        template = '<div style="">{}</div>'
        return template.format(concat_svgs)

In [7]:
HorizontalDisplay(strengths)

In [8]:
def mol_vector_from_file(file_path):
    """Accepts a file path to an SDF/MOL file and generates a mol vector.
    """
    rdkit_mol = rdkit.Chem.MolFromMolFile(file_path)
    rdkit_mol = rdkit.Chem.AddHs(rdkit_mol, addCoords=True)
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    mol_vector = helpers.sum_atomic_map(atomic_map)
    mol_vector["name"] = file_path.split("/")[-1]
    return mol_vector

In [9]:
# Build molecular vectors
vects = list()
for f in files:
    file_path = os.path.join(cdk2_path, f)
    vects.append(mol_vector_from_file(file_path))
vects

[{'sdc': 7.3073, 'sdx': 4.6706, 'sa': 5.5396, 'name': '20Z.sdf'},
 {'sdc': 5.585, 'sdx': 6.2794, 'sa': 6.4553, 'name': '26Z.sdf'},
 {'sdc': 5.3399, 'sdx': 2.6102, 'sa': 3.4443, 'name': 'X02.sdf'},
 {'sdc': 5.489, 'sdx': 2.6952, 'sa': 4.0838, 'name': 'X35.sdf'},
 {'sdc': 5.6043, 'sdx': 2.5862, 'sa': 3.4162, 'name': 'X36.sdf'},
 {'sdc': 6.7521, 'sdx': 6.9577, 'sa': 7.407, 'name': 'X44.sdf'}]

In [10]:
# Export renderings
# for f in files:
#     file_path = os.path.join(cdk2_path, f)
#     output_filepath = f.split(".")[0] + ".png"
#     svg2png(bytestring=atomistic_strength_from_file(file_path), write_to=output_filepath)

In [11]:
def charge_distribution_from_file(file_path, evaluate_idxs):
    """Accepts a file path to an SDF/MOL file and generates a charge distribution rendering.
    """
    rdkit_mol = rdkit.Chem.MolFromMolFile(file_path)
    rdkit_mol = rdkit.Chem.AddHs(rdkit_mol, addCoords=True)
    kallisto_mol = core.kallisto_molecule_from_rdkit_molecule(rdkit_mol)
    atoms_and_nbrs = core.get_covalent_atom_idxs(rdkit_mol)
    charges = core.get_charges_from_kallisto_molecule(kallisto_mol, charge=0)
    atomic_map = core.calculate_polar_strength_map(rdkit_mol, kallisto_mol, atoms_and_nbrs, charges)
    rdkit.Chem.rdDepictor.Compute2DCoords(rdkit_mol)    # Flatten molecule

    # Add partial charge label to selected indices
    for a in rdkit_mol.GetAtoms():
        idx = a.GetIdx()
        if idx in evaluate_idxs:
            lbl = '%.2f'%(atomic_map[idx]["eeq"])
            a.SetProp('atomNote', str(lbl))
        d2d = rdMolDraw2D.MolDraw2DSVG(500, 500)
        d2d.drawOptions().annotationFontScale = 0.7
        d2d.DrawMolecule(rdkit_mol)
        d2d.FinishDrawing()

    # Add ligand name as a title
    img_text = d2d.GetDrawingText()
    img_text = img_text.replace('svg:','')
    fig = sg.fromstring(img_text)
    label = sg.TextElement(250, 15, file_path.split("/")[-1], size=14, 
                        font='sans-serif', anchor='middle', color='#000000')
    fig.append(label)
    img_text = fig.to_str().decode("utf-8")
    return img_text

In [12]:
# Build charge distribution visualisations for selected atom indices
q_dist_mols = {"X02.sdf": [24, 25, 9, 6, 7, 10, 17, 11, 12, 13, 14, 15, 16],
               "X35.sdf": [18, 19, 2, 1, 3, 4, 11, 5, 6, 7, 8, 9, 10]}
dists = list()
for k, v in q_dist_mols.items():
    for f in files:
        if k == f:
            file_path = os.path.join(cdk2_path, f)
            dists.append(charge_distribution_from_file(file_path, v))
HorizontalDisplay(dists)

In [13]:
# Export renderings
# for k, v in q_dist_mols.items():
#     for f in files:
#         if k == f:
#             file_path = os.path.join(cdk2_path, f)
#             output_filepath = f.split(".")[0] + ".png"
#             svg2png(bytestring=charge_distribution_from_file(file_path, v), write_to=output_filepath)